In [2]:
#pip install paddlepaddle


In [3]:
# pip install paddleocr

In [36]:
# pip install transformers torch numpy

In [5]:
import pytesseract
from PIL import Image
import easyocr
# with torch.amp.autocast('cuda'):
reader = easyocr.Reader(['ko', 'en'])  # 한글과 영어 설정
result = reader.readtext('/kaggle/input/posterimage/.png')
for r in result:
    print(r[1])  # 추출된 텍스트 출력

    #빠르지만 정확도가 다소 떨어짐

/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/parallel_apply.py:79: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.device(device), torch.cuda.stream(stream), autocast(enabled=autocast_enabled):


00:10
LTE
83
[서울]소비자가바꾸스세상(0713-001)
과목공지 (조회)
제목: 여러분 공모전 안내입니다
계시일: 어제 오후 213
여러분! 안녕하세요?
유난히 무더워던 여름도 지나가고 선선한 가을을 기다리면서 .
공모전올 안내해 드립니다
도전 바라면서 .. 철부파일도 올리니 참고해 주세요
혹여 도움이 필요하면 연락주세요.
김시월 드림
2024 시니어 소비자교육 존렌츠 공모전-
주제: 디지털 소비환경에서의 시니어 소비자교육 소렌츠
예시: 인터넷 쇼풍, 홍소굉 이용 방법
인터넷 금움거래 방법
모바일 입 이용하여 거래하는 방법 등
형식: 동영상(2-3분) , 카드뉴스(8-10장)
대상: 대학(원)생(휴학생 포함) (개인 또는 4명 이내 팀 가능)
제출 마감: 2024년 11월 29일, 온라인으로 파일제출
상금: 총 상금 500만원
0
2024 소비자교육 밖렌l공모전 포스터 최종pdf (1.17 MB)
대글 0
모로


In [40]:

# 정확도는 더 높으나 속도가 조금 느리다.
# 필요한 라이브러리 임포트
import paddleocr
import logging

# 로그 레벨 설정
logging.getLogger("ppocr").setLevel(logging.WARNING)

# 한글과 영어 추출을 위한 OCR 객체 생성
ocr = paddleocr.PaddleOCR(lang='korean')

# 이미지에서 텍스트 추출
result = ocr.ocr('/kaggle/input/posterimage/.png', cls=True)

# 텍스트를 줄별로 저장할 리스트
extracted_lines = []

# y좌표의 범위 설정 (오차)
y_threshold = 30  # 예: 10픽셀 이내의 차이

# 각 줄의 텍스트를 저장할 변수
line_text = ""

# 이전 y좌표를 저장하기 위한 변수
previous_y = None

# 결과를 순회하며 줄 텍스트를 구성
for line in result:
    for word in line:
        word_text = word[1][0]  # 단어 텍스트
        y_coordinate = (word[0][0][1] + word[0][2][1]) / 2  # 단어의 y좌표 (중간값)

        # 첫 단어일 경우 또는 이전 y좌표가 None인 경우
        if previous_y is None:
            line_text += word_text + " "
            previous_y = y_coordinate
        else:
            # 이전 y좌표와 현재 y좌표의 차이 계산
            if abs(y_coordinate - previous_y) <= y_threshold:
                line_text += word_text + " "
            else:
                # y좌표 차이가 threshold를 넘으면 현재 줄을 리스트에 추가하고 line_text 초기화
                extracted_lines.append(line_text.strip())  # 앞뒤 공백 제거하여 리스트에 추가
                line_text = word_text + " "  # 새로운 줄 시작
                previous_y = y_coordinate  # 이전 y좌표 업데이트

# 마지막으로 남아있는 줄 텍스트 추가
if line_text:
    extracted_lines.append(line_text.strip())

# 최종적으로 추출된 텍스트 리스트 출력
print(extracted_lines)

[2024/10/16 09:10:41] ppocr WARNING: Since the angle classifier is not initialized, it will not be used during the forward process
['00:10 11 LTE 83', '[서울]소비자가바꾸는세상0713-001', '과목공지 조회', '제목 :여러분! 공모전 안내입니다', '게시일 :어제오후2:13', '여러분! 안녕하세요?', '유난히 무더웠던 여름도 지나가고 선선한 가을을 기다리면서', '공모전을 안내해 드립니다', '바라면서첨부파일도 도전 올리니 참고해 주세요', '혹여 도움이 필요하면 연락주세요', '김시월 드림', '2024 시니어 소비자교육 콘텐츠 공모전-', '주제: 디지털 소비환경에서의 시니어 소비자교육 콘텐츠', '예시: 인터넷 쇼핑 홈쇼핑 이용 방법', '인터넷 금융거래 방법', '모바일 앱 이용하여 거래하는 방법동 등', '형식: 동영상2-3분 카드뉴스8-10장', '대상: 대학원생휴학생 포함개인 또는 4명 이내 팀 가능', '제출 마감: 2024년 11월 29일 온라인으로파일제출', '상금: 총상금 500만원', '1 2024 소비자교육 콘텐츠공모전 포스터 최종pdf 117 MB', '덧글 0', '모로']


In [39]:
#정규 표현식으로 찾기
import re
all_dates = []
date_pattern = r"\d{4}년 \d{1,2}월 \d{1,2}일"
for line in extracted_lines:
    dates = re.findall(date_pattern, line)
    all_dates.extend(dates)

# 찾은 날짜 출력
print(all_dates)

['2024년 11월 29일']


In [13]:
# from transformers import AutoModelForCausalLM, AutoTokenizer
# import torch

# # 모델과 토크나이저 로드
# model_name = "yanolja/EEVE-Korean-10.8B-v1.0"
# tokenizer = AutoTokenizer.from_pretrained(model_name)
# model = AutoModelForCausalLM.from_pretrained(model_name)

# # GPU 사용 여부에 따른 설정
# device = "cuda" if torch.cuda.is_available() else "cpu"
# model.to(device)

# # 입력 텍스트
# input_text = "안녕하세요, 오늘 날씨는 어떤가요?"
# input_ids = tokenizer.encode(input_text, return_tensors="pt").to(device)

# # 텍스트 생성
# with torch.no_grad():
#     output = model.generate(input_ids, max_length=100, num_return_sequences=1)

# # 생성된 텍스트 출력
# generated_text = tokenizer.decode(output[0], skip_special_tokens=True)
# print(generated_text)


In [16]:
# from transformers import pipeline
# #NER모델로 뽑아보기 안된다..
# # NER 모델 파이프라인 생성
# # GPU 사용 설정
# nlp = pipeline("ner", model="dbmdz/bert-large-cased-finetuned-conll03-english", device=0)

# # 텍스트에 대해 네임드 엔티티 인식(NER) 수행
# results = nlp(extracted_text)

# # 일정과 관련된 엔티티(날짜, 제목) 필터링
# events = [(entity['word'], entity['entity']) for entity in results if entity['entity'] in ['DATE', 'EVENT']]

# # 결과 출력
# print("추출된 이벤트 및 날짜:")
# for event in events:
#     print(event)

Some weights of the model checkpoint at dbmdz/bert-large-cased-finetuned-conll03-english were not used when initializing BertForTokenClassification: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


추출된 이벤트 및 날짜:


In [22]:
#kor electra model 로드 
from transformers import ElectraTokenizer, ElectraForTokenClassification
import torch

# ELECTRA 모델과 토크나이저 로드
model_name = "kykim/electra-kor-base"  # ELECTRA 기반 한국어 모델 이름
tokenizer = ElectraTokenizer.from_pretrained(model_name)
model = ElectraForTokenClassification.from_pretrained(model_name)

tokenizer_config.json:   0%|          | 0.00/80.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/344k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/620 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/473M [00:00<?, ?B/s]

Some weights of ElectraForTokenClassification were not initialized from the model checkpoint at kykim/electra-kor-base and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [33]:
# 텍스트를 토큰화하고 텐서로 변환
inputs = tokenizer(extracted_lines, return_tensors="pt")

# 모델 예측
with torch.no_grad():
    outputs = model(**inputs)
    logits = outputs.logits

# 예측된 클래스 (엔티티 태그) 결정
predicted_classes = torch.argmax(logits, dim=2)

# 결과 출력
for token, pred in zip(inputs['input_ids'][0], predicted_classes[0]):
    print(tokenizer.decode(token), pred.item())


ValueError: Unable to create tensor, you should probably activate truncation and/or padding with 'padding=True' 'truncation=True' to have batched tensors with the same length. Perhaps your features (`input_ids` in this case) have excessive nesting (inputs type `list` where type `int` is expected).

In [24]:
# 예시 엔티티 필터링 (사용할 엔티티에 맞게 수정)
entities = [(tokenizer.decode(token), label.item()) for token, label in zip(inputs['input_ids'][0], predicted_classes[0]) if label in [0, 1]]  # 0과 1은 예시입니다.

# 결과 출력
for entity in entities:
    print(entity)


('[CLS]', 1)
('00', 0)
(':', 0)
('10', 0)
('11', 0)
('lte', 0)
('83', 0)
('[', 0)
('서울', 0)
(']', 0)
('소비자가', 0)
('##바', 0)
('##꾸', 0)
('##는', 0)
('##세상', 0)
('##0', 0)
('##71', 0)
('##3', 0)
('-', 0)
('00', 0)
('##1', 0)
('과목', 0)
('##공', 0)
('##지', 0)
('조회', 0)
('제목', 0)
(':', 0)
('여러분', 0)
('!', 0)
('공모', 0)
('##전', 0)
('안내', 0)
('##입니다', 0)
('게시', 0)
('##일', 0)
(':', 0)
('어제', 0)
('##오', 0)
('##후', 0)
('##2', 0)
(':', 0)
('13', 0)
('여러분', 0)
('!', 0)
('안녕하세요', 0)
('?', 0)
('유난히', 0)
('무더', 0)
('##웠던', 0)
('여름', 0)
('##도', 0)
('지나가고', 0)
('선선', 0)
('##한', 0)
('가을', 0)
('##을', 0)
('기다리', 0)
('##면서', 0)
('공모', 0)
('##전을', 0)
('안내', 0)
('##해', 0)
('드립니다', 0)
('바라', 0)
('##면서', 0)
('##첨', 0)
('##부', 0)
('##파일', 0)
('##도', 0)
('도전', 0)
('올리', 0)
('##니', 0)
('참고해', 0)
('주세요', 0)
('혹여', 0)
('도움이', 0)
('필요', 0)
('##하면', 0)
('연락주세요', 0)
('김', 0)
('##시', 0)
('##월', 0)
('드림', 0)
('202', 0)
('##4', 0)
('시', 0)
('##니어', 0)
('소비자', 0)
('##교육', 0)
('콘텐츠', 0)
('공모', 0)
('##전', 0)
('-', 0)
('주제', 0)